In [ ]:
import cv2
import mediapipe as mp

In [ ]:
def initialize_mediapipe():
    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    return mp_hands, mp_drawing, hands

def procesar_frame(frame, hands):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = hands.process(image)
    return results

def calcular_diferencia(puntos_previos,puntos_actuales ):
    if (puntos_previos is None & puntos_actuales is not None) or (puntos_actuales is None & puntos_previos is not None):
        return float('inf')

    num_hands_previas = len(puntos_previos)
    num_hands_actuales = len(puntos_actuales)

    if num_hands_previas != num_hands_actuales:
        return float('inf')

    total_diff = 0

    for i in range(num_hands_previas):
        hand_prev = puntos_previos[i].landmark
        hand_actual = puntos_actuales[i].landmark

        diff = sum([
            abs(lm1.x - lm2.x) + abs(lm1.y - lm2.y) + abs(lm1.z - lm2.z)
            for lm1, lm2 in zip(hand_prev, hand_actual)
        ])
        total_diff += diff

    if num_hands_previas >= 2:
        total_diff /= num_hands_previas

def extract_key_frames(video_path, threshold=0.1):
    cap = cv2.VideoCapture(video_path)
    mp_hands,mp_drawing, hands=initialize_mediapipe()
    puntos_previos = None
    key_frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        results = procesar_frame(frame, hands)
        if results.multi_hand_landmarks:
            puntos_actuales = results.multi_hand_landmarks
            for hand_landmarks in puntos_actuales:
                mp_drawing.draw_landmarks(
                    frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(255,255,0), thickness=4, circle_radius=5),
                    mp_drawing.DrawingSpec(color=(255,0,255), thickness=4))
                #no sé como mostrarlo

            diff = calcular_diferencia(puntos_previos, puntos_actuales)

            if diff > threshold:
                key_frames.append(frame)
                puntos_previos = puntos_actuales
        else:
            puntos_previos = None

    cap.release()
    return key_frames
def save_key_frames(key_frames, output_folder='key_frames'):
    for idx, key_frame in enumerate(key_frames):
        cv2.imwrite(f'{output_folder}/key_frame_{idx}.png', key_frame)

In [ ]:
video_path = 'ruta_del_video.mp4'
key_frames = extract_key_frames(video_path)
save_key_frames(key_frames)